In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)




In [ ]:
!pip install torch torchvision feather-format kornia pyarrow --upgrade   > /dev/null
!pip install git+https://github.com/fastai/fastai2                 > /dev/null

In [ ]:
from fastai2.torch_basics import *
from fastai2.test import *
from fastai2.layers import *
from fastai2.data.all import *
from fastai2.optimizer import *
from fastai2.learner import *
from fastai2.metrics import *
from fastai2.vision.all import *
from fastai2.vision.learner import *
from fastai2.vision.models import *
from fastai2.callback.all import *
from fastai2.basics           import *
from fastai2.vision.all       import *
from fastai2.medical.imaging  import *


In [ ]:
np.set_printoptions(linewidth=120)
matplotlib.rcParams['image.cmap'] = 'bone'
set_seed(42)
set_num_threads(1)

In [ ]:

path = Path('../input/rsna-intracranial-hemorrhage-detection/rsna-intracranial-hemorrhage-detection/')
path_train = path/'stage_2_train'
path_test = path/'stage_2_test'


path_dest = Path()
path_dest.mkdir(exist_ok=True)

path_inp = Path('../input')
path_df = path_inp/'rsna-stage2-meta'


In [ ]:
path_df.ls()

In [ ]:
df_lbls = pd.read_feather(path_df/'labels_stage_2.fth')
df_tst = pd.read_feather(path_df/'fns_test_2.fth')
df_trn = pd.read_feather(path_df/'fns_train_2.fth').dropna(subset=['img_pct_window'])
comb = df_trn.join(df_lbls.set_index('ID'), 'SOPInstanceUID')

** Fix incorrect RescaleIntercept ** 



In [ ]:
repr_flds = ['BitsStored','PixelRepresentation']
df1 = comb.query('(BitsStored==12) & (PixelRepresentation==0)')
df2 = comb.query('(BitsStored==12) & (PixelRepresentation==1)')
df3 = comb.query('BitsStored==16')
dfs = L(df1,df2,df3)

In [ ]:
dfs[0]

In [ ]:
dfs[1]

In [ ]:
dfs[2]

In [ ]:
def df2dcm(df): 
    return L(Path(o).dcmread() for o in df.fname.values)

In [ ]:
df_iffy = df1[df1.RescaleIntercept>-1000]
dcms = df2dcm(df_iffy)

_,axs = subplots(4,4, imsize=3)
for i,ax in enumerate(axs.flat): dcms[i].show(ax=ax)

In [ ]:
dcm = dcms[2]
d = dcm.pixel_array
plt.hist(d.flatten());

In [ ]:
len(dcms)

In [ ]:
d1 = df2dcm(df1.iloc[[0]])[0].pixel_array
plt.hist(d1.flatten());


In [ ]:
scipy.stats.mode(d.flatten()).mode[0]

In [ ]:
d += 1000

px_mode = scipy.stats.mode(d.flatten()).mode[0]
d[d>=px_mode] = d[d>=px_mode] - px_mode
dcm.PixelData = d.tobytes()
dcm.RescaleIntercept = -1000


In [ ]:
plt.hist(dcm.pixel_array.flatten());

In [ ]:
_,axs = subplots(1,2)
dcm.show(ax=axs[0]);   
dcm.show(dicom_windows.brain, ax=axs[1])


In [ ]:
def fix_pxrepr(dcm):
    if dcm.PixelRepresentation != 0 or dcm.RescaleIntercept<-1000: return
    x = dcm.pixel_array + 1000
    px_mode = 4096
    x[x>=px_mode] = x[x>=px_mode] - px_mode
    dcm.PixelData = x.tobytes()
    dcm.RescaleIntercept = -1000

In [ ]:
dcms = df2dcm(df_iffy)
dcms.map(fix_pxrepr)

_,axs = subplots(2,5, imsize=3)
for i,ax in enumerate(axs.flat): dcms[i].show(ax=ax)

Remove useless images
* Our goal here is to create a small, fast, convenient dataset for rapid prototyping. So let's get rid of images that don't provide much useful information, such as those with very little actual brain tissue in them. Brain tissue is in the region (0,80). Let's find out how many pixels in this region are in each image. When we created the metadata data frame, we got a img_pct_window column included which has the % of pixels in the brain window.

In [ ]:
df_iffy.img_pct_window[:10].values

In [ ]:
plt.hist(comb.img_pct_window,40);

There are a lot of images with nearly no brain tissue in them - presumably they're the slices above and below the brain. Let's see if they have any labels:

In [ ]:
comb = comb.assign(pct_cut = pd.cut(comb.img_pct_window, [0,0.02,0.05,0.1,0.2,0.3,1]))
comb.pivot_table(values='any', index='pct_cut', aggfunc=['sum','count']).T

We can see that, as expected, the images with little brain tissue (<2% of pixels) have almost no labels. So let's remove them. (Interestingly, we can also see a strong relationship between these two variables.)

In [ ]:
comb.drop(comb.query('img_pct_window<0.02').index, inplace=True)

Resample to 2/3 split
-  keep every row with a label:
-  we'll keep half that number of images without a label, which should keep the resultant size under Kaggle's 20GB dataset limit:



In [ ]:
df_lbl = comb.query('any==True')
n_lbl = len(df_lbl)
n_lbl

In [ ]:
df_nonlbl = comb.query('any==False').sample(n_lbl//2)
len(df_nonlbl)

In [ ]:
comb = pd.concat([df_lbl,df_nonlbl])
len(comb)


In [ ]:
comb.head()

In [ ]:
comb_ = comb.reset_index()
comb_.index

In [ ]:
comb_ = comb_.drop(['pct_cut'],axis =1)

In [ ]:
comb_.to_feather('comb_stage_2.fth')


In [ ]:
from IPython.display import FileLink, FileLinks
FileLink('comb_stage_2.fth')

Crop to just brain area
- To create a smaller and faster dataset, we'll need smaller images. So let's make sure they contain the important information, by cropping out the non-brain area. To do so, we start with an image like this:

In [ ]:
dcm = Path(dcms[3].filename).dcmread()


In [ ]:
fix_pxrepr(dcm)
px = dcm.windowed(*dicom_windows.brain)
show_image(px);

In [ ]:
blurred = gauss_blur2d(px, 100)
show_image(blurred);

In [ ]:
show_image(blurred>0.3)

In [ ]:
dcm.show(dicom_windows.brain)
show_image(dcm.mask_from_blur(dicom_windows.brain), cmap=plt.cm.Reds, alpha=0.6)

We can use fastai's mask_from_blur method to do this for us. We'll overlay the results on a few images to see if it looks OK:

In [ ]:
_,axs = subplots(1,4, imsize=3)
for i,ax in enumerate(axs.flat):
    dcms[i].show(dicom_windows.brain, ax=ax)
    show_image(dcms[i].mask_from_blur(dicom_windows.brain), cmap=plt.cm.Reds, alpha=0.6, ax=ax)

In [ ]:
def pad_square(x):
    r,c = x.shape
    d = (c-r)/2
    pl,pr,pt,pb = 0,0,0,0
    if d>0: pt,pd = int(math.floor( d)),int(math.ceil( d))        
    else:   pl,pr = int(math.floor(-d)),int(math.ceil(-d))
    return np.pad(x, ((pt,pb),(pl,pr)), 'minimum')

def crop_mask(x):
    mask = x.mask_from_blur(dicom_windows.brain)
    bb = mask2bbox(mask)
    if bb is None: return
    lo,hi = bb
    cropped = x.pixel_array[lo[0]:hi[0],lo[1]:hi[1]]
    x.pixel_array = pad_square(cropped)

In [ ]:
_,axs = subplots(1,2)
dcm.show(ax=axs[0])
crop_mask(dcm)
dcm.show(ax=axs[1]);

In [ ]:
htypes = 'any','epidural','intraparenchymal','intraventricular','subarachnoid','subdural'

def get_samples(df):
    recs = [df.query(f'{c}==1').sample() for c in htypes]
    recs.append(df.query('any==0').sample())
    return pd.concat(recs).fname.values

sample_fns = concat(*dfs.map(get_samples))
sample_dcms = tuple(Path(o).dcmread().scaled_px for o in sample_fns)
samples = torch.stack(sample_dcms)
bins = samples.freqhist_bins()

In [ ]:
(path_dest/'bins_2.pkl').save(bins)

In [ ]:
from IPython.display import FileLink, FileLinks
FileLink('bins_2.pkl')